# Collecting Reddit Data with PRAW

This notebook contains examples for using web-based APIs (Application Programmer Interfaces) to download data from social media platforms.

This notebook focuses specifically on _Reddit_.

For most services, we need to register with the platform in order to use their API.
Instructions for the registration processes are outlined in each specific section below.

We will use APIs because they *can* be much faster than manually copying and pasting data from the web site, APIs provide uniform methods for accessing resources (searching for keywords, places, or dates), and it should conform to the platform's terms of service (important for partnering and publications).
Note however that each of these platforms has strict limits on access times: e.g., requests per hour, search history depth, maximum number of items returned per request, and similar.

In [ ]:
%matplotlib inline

import json

<hr>

## Reddit API

Reddit's API used to be the easiest to use since it did not require credentials to access data on its subreddit pages.
Unfortunately, this process has been changed, and developers now need to create a Reddit application on Reddit's app page located here: (https://www.reddit.com/prefs/apps/).

In [ ]:
# For our first piece of code, we need to import the package 
# that connects to Reddit. Praw is a thin wrapper around reddit's 
# web APIs and works well

import praw

### Creating a Reddit Application
Go to https://www.reddit.com/prefs/apps/.
Scroll down to "create application", select "web app", and provide a name, description, and URL (which can be anything).

After you press "create app", you will be redirected to a new page with information about your application. Copy the unique identifiers below "web app" and beside "secret". These are your client_id and client_secret values, which you need below.

In [ ]:
# Now we specify a "unique" user agent for our code
# This is primarily for identification, I think, and some
# user-agents of bad actors might be blocked
redditApi = praw.Reddit(client_id='xxx',
                        client_secret='xxx',
                        user_agent='is688_cbuntain_v01')

### Capturing Reddit Posts

Now for a given subreddit, we can get the newest posts to that sub. 
Post titles are generally short, so you could treat them as something similar to a tweet.

In [ ]:
subreddit = "worldnews"

targetSub = redditApi.subreddit(subreddit)

submissions = targetSub.new(limit=10)
for post in submissions:
    print(post.title, post.author.name)

In [ ]:
subreddit = "equality"
targetSub = redditApi.subreddit(subreddit)

# Array for storing subreddit post titles
title_list = []
submissions = targetSub.new(limit=10)
for post in submissions:
        title_list.append(post.title)

print(title_list)


from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()
for post_title in title_list:
    tokenizer.tokenize(post_title)

### Leveraging Reddit's Voting

Getting the new posts gives us the most up-to-date information. 
You can also get the "hot" posts, "top" posts, etc. that should be of higher quality. 
In theory.
__Caveat emptor__

In [ ]:
subreddit = "worldnews"

targetSub = redditApi.subreddit(subreddit)

submissions = targetSub.hot(limit=5)
for post in submissions:
    print(post.title)

### Following Multiple Subreddits

Reddit has a mechanism called "multireddits" that essentially allow you to view multiple reddits together as though they were one.
To do this, you need to concatenate your subreddits of interesting using the "+" sign.

In [ ]:
subreddit = "worldnews+news"

targetSub = redditApi.subreddit(subreddit)
submissions = targetSub.new(limit=10)
for post in submissions:
    print(post.title, post.author)

### Accessing Reddit Comments

While you're never supposed to read the comments, for certain live streams or new and rising posts, the comments may provide useful insight into events on the ground or people's sentiment.
New posts may not have comments yet though.

Comments are attached to the post title, so for a given submission, you can pull its comments directly.

Note Reddit returns pages of comments to prevent server overload, so you will not get all comments at once and will have to write code for getting more comments than the top ones returned at first.
This pagination is performed using the MoreXYZ objects (e.g., MoreComments or MorePosts).

In [ ]:
subreddit = "worldnews"

breadthCommentCount = 5

targetSub = redditApi.subreddit(subreddit)

submissions = targetSub.hot(limit=1)

for post in submissions:
    print (post.title)
    
    post.comment_limit = breadthCommentCount
    
    # Get the top few comments
    for comment in post.comments.list():
        if isinstance(comment, praw.models.MoreComments):
            continue
        
        print ("---", comment.name, "---")
        print ("\t", comment.body)
        
        for reply in comment.replies.list():
            if isinstance(reply, praw.models.MoreComments):
                continue
            
            print ("\t", "---", reply.name, "---")
            print ("\t\t", reply.body)

### Other Functionality

Reddit has a deep comment structure, and the code above only goes two levels down (top comment and top comment reply). 
You can view Praw's additional functionality, replete with examples on its website here: http://praw.readthedocs.io/